In [ ]:
# Based on https://github.com/computervisioneng/image-segmentation-yolov8/

### Create YOLO labels

In [ ]:
import os
import cv2

from google.colab import drive
drive.mount('/content/drive')

input_dir = 'drive/MyDrive/Master Geo Information Science/Internship/Data/Ready/Ready/Masks'
output_dir = 'drive/MyDrive/Master Geo Information Science/Internship/Data/Ready/Ready/yolo_labels'
os.makedirs(output_dir, exist_ok = True)

for j in os.listdir(input_dir):
    image_path = os.path.join(input_dir, j)
    # load the binary mask and get its contours
    mask = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

    H, W = mask.shape
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # convert the contours to polygons
    polygons = []
    for cnt in contours:
        if cv2.contourArea(cnt) > 200:
            polygon = []
            for point in cnt:
                x, y = point[0]
                polygon.append(x / W)
                polygon.append(y / H)
            polygons.append(polygon)

    # print the polygons
    with open('{}.txt'.format(os.path.join(output_dir, j)[:-4]), 'w') as f:
        for polygon in polygons:
            for p_, p in enumerate(polygon):
                if p_ == len(polygon) - 1:
                    f.write('{}\n'.format(p))
                elif p_ == 0:
                    f.write('0 {} '.format(p))
                else:
                    f.write('{} '.format(p))

        f.close()

### Create config

In [ ]:
import shutil
import os
import random

# Create folder structure (delete and create new if already exists)
img_dir = 'drive/MyDrive/Master Geo Information Science/Internship/Data/Ready/Ready/Images'
label_dir = 'drive/MyDrive/Master Geo Information Science/Internship/Data/Ready/Ready/yolo_labels'
yolo_dir = 'drive/MyDrive/Master Geo Information Science/Internship/Data/Ready/Ready/YOLO'

if os.path.exists(os.path.join(yolo_dir, "train/images")):
    shutil.rmtree(os.path.join(yolo_dir, "train/images"))
os.makedirs(os.path.join(yolo_dir, "train/images"))
if os.path.exists(os.path.join(yolo_dir, "train/labels")):
    shutil.rmtree(os.path.join(yolo_dir, "train/labels"))
os.makedirs(os.path.join(yolo_dir, "train/labels"))
if os.path.exists(os.path.join(yolo_dir, "val/images")):
    shutil.rmtree(os.path.join(yolo_dir, "val/images"))
os.makedirs(os.path.join(yolo_dir, "val/images"))
if os.path.exists(os.path.join(yolo_dir, "val/labels")):
    shutil.rmtree(os.path.join(yolo_dir, "val/labels"))
os.makedirs(os.path.join(yolo_dir, "val/labels"))

# Shuffle images and labels similarly (important to sort and list by file type --> don't get the folders)
image_list = sorted([file for file in os.listdir(img_dir) if file.lower().endswith(('.jpg', '.jpeg', '.png'))])
label_list = sorted([file for file in os.listdir(label_dir) if file.lower().endswith(('.txt'))])

both_lists = list(zip(image_list, label_list))
random.shuffle(both_lists)
image_list, label_list = zip(*both_lists)

# Copy a ratio to train and val folders
split_ratio = 0.9
img_split_index = int(len(image_list) * split_ratio)
label_split_index = int(len(label_list) * split_ratio)

n = 1
# Copy train
for img_file, label_file in zip(image_list[:img_split_index], label_list[:label_split_index]):
    shutil.copy(os.path.join(img_dir, img_file), os.path.join(yolo_dir, "train/images", "img_"+str(n)+".png"))
    shutil.copy(os.path.join(label_dir, label_file), os.path.join(yolo_dir, "train/labels", "img_"+str(n)+".txt"))
    n = n + 1

# Copy val
for img_file, label_file in zip(image_list[img_split_index:], label_list[label_split_index:]):
    shutil.copy(os.path.join(img_dir, img_file), os.path.join(yolo_dir, "val/images", "img_"+str(n)+".png"))
    shutil.copy(os.path.join(label_dir, label_file), os.path.join(yolo_dir, "val/labels", "img_"+str(n)+".txt"))
    n = n + 1


In [ ]:
# Create YAML file

!pip install pyyaml

import yaml

data = {
    'path': '/content/drive/MyDrive/Master Geo Information Science/Internship/Data/Ready/Ready/', # root path, start at home directory
    'train': 'YOLO/train/images', # Labels will automatically be found (?)
    'val': 'YOLO/val/images',
    'nc': 1, # number of classes
    'names': ['bird']
}

with open('drive/MyDrive/Master Geo Information Science/Internship/Data/Ready/Ready/config.yaml', 'w') as file:
    yaml.dump(data, file, default_flow_style=False)


### Train YOLO

In [ ]:
!pip install ultralytics
!pip install opencv-python

In [ ]:
from ultralytics import YOLO
import yaml

from google.colab import drive
drive.mount('/content/drive')

# load a pretrained model (recommended for training)
model = YOLO('yolov8n-seg.pt')

# arguments for training (see https://docs.ultralytics.com/modes/train/#arguments)
args = {"data": 'drive/MyDrive/Master Geo Information Science/Internship/Data/Ready/Ready/config.yaml',
        "epochs": 5, "batch": 3, "imgsz": 150,
        "project": '/content/drive/MyDrive/Master Geo Information Science/Internship/Data/Model/YOLO', "name": "Vogels", "exist_ok": True} # where to save model

# train
model.train(**args)

Ultralytics YOLOv8.0.211 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=drive/MyDrive/Master Geo Information Science/Internship/Data/Ready/Ready/config.yaml, epochs=5, patience=50, batch=3, imgsz=150, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/Master Geo Information Science/Internship/Data/Model/YOLO, name=Vogels, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, 

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x791da833aad0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041, 

### Save model (traditionally)


In [ ]:
!pip3 install --upgrade ultralytics

In [ ]:
import torch

# Models and predictions are saved in runs/segment/
# model = YOLO('/content/runs/segment/train3/weights/best.pt')

# save in PyTorch format
# torch.save(model, '/content/drive/MyDrive/Master Geo Information Science/Internship/Data/Model/YOLO/Vogels/model.pt')
# torch.save(model.state_dict(), '/content/drive/MyDrive/Master Geo Information Science/Internship/Data/Model/YOLO/Vogels/weights.pt')

# save in other platforms or languages:
# read https://docs.ultralytics.com/modes/export/
# model.export(format='onnx')

### Test inference (images and results are also saved in model folder)

In [ ]:
from ultralytics import YOLO
import torch
from PIL import Image
import cv2
import os, random
import matplotlib.pyplot as plt
import numpy as np

# Load model
model = YOLO('/content/drive/MyDrive/Master Geo Information Science/Internship/Data/Model/YOLO/Vogels/weights/best.pt')

In [ ]:
# Image folder with images to test
image_folder = 'drive/MyDrive/Master Geo Information Science/Internship/Data/Ready/Ready/YOLO/val/images/'

# List all image files in the folder
image_files = [f for f in os.listdir(image_folder) if f.endswith('.png')]

# Select 10 random images
random_images = random.sample(image_files, 10)

# Iterate over the selected images
for image_file in random_images:
    image_path = os.path.join(image_folder, image_file)

    # Get predictions for the current image
    result = model(image_path)

    for r in result:
        # print(r)

        im_array = r.plot()  # plot a BGR numpy array of predictions
        im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image

        # Display the image
        plt.imshow(im)
        plt.show()

Output hidden; open in https://colab.research.google.com to view.

## SAHI

In [ ]:
# Download latest versions
!pip install -U sahi
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cpu/torch1.10/index.html # for Detectron2-cpu
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html # for Detectron2-cuda11.1
# !pip install 'git+https://github.com/facebookresearch/detectron2.git'

# will be used for detectron2 fasterrcnn model zoo name
from sahi.utils.detectron2 import Detectron2TestConstants
from sahi.utils.yolov8 import (
    download_yolov8s_model,
)

# import required functions, classes
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction, predict, get_prediction
from sahi.utils.file import download_from_url
from sahi.utils.cv import read_image
from IPython.display import Image

import os
os.getcwd()

'/content'

In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

detection_model = AutoDetectionModel.from_pretrained(
    model_type='yolov8',
    model_path= '/content/drive/MyDrive/Master Geo Information Science/Internship/Data/Model/YOLO/Vogels/weights/best.pt', # example: model_weights.pt
    config_path=os.path.join('/content/drive/MyDrive/Master Geo Information Science/Internship/Data/Model/YOLO/Vogels/args.yaml'), # example: model_cfg.yaml
    confidence_threshold=0.5,
    image_size=640,
    device= 'cuda:0' # "cpu"
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Test on a subset raster

In [ ]:
import rasterio
from rasterio.windows import from_bounds
from rasterio.windows import Window
from rasterio.transform import from_origin
from rasterio.transform import Affine

import numpy as np
from osgeo import gdal

import matplotlib.pyplot as plt

# Create subset
raster_data_path = "/content/drive/MyDrive/Master Geo Information Science/Internship/Data/Test rasters/WaterdunenWestsubset.tif"
out_path = "/content/drive/MyDrive/Master Geo Information Science/Internship/Data/Test rasters/subset.tif"
out_img = "/content/drive/MyDrive/Master Geo Information Science/Internship/Data/Test rasters/subset.png"

# Open the original raster file
with rasterio.open(raster_data_path) as src:
    # Get the dimensions of the original raster
    width = src.width
    height = src.height

    # Calculate the bounding box coordinates (check what gives nice subset)
    xmin = (width / 10) * 2
    ymin = (height / 10) * 4
    xmax = (width / 10) * 3
    ymax = (height / 10) * 5
    # xmin = width // 10
    # ymin = height // 2
    # xmax = width // 3
    # ymax = height // 1.5

    print(xmin, ymin, xmax, ymax)

    # Define a window using these coordinates
    window = Window(xmin, ymin, xmax - xmin, ymax - ymin)

    # Read the data within the window
    quarter_data = src.read(window=window)

    # Update the transform to reflect the new window
    new_transform = src.window_transform(window)

    # Create a new raster file with the quarter data
    with rasterio.open(out_path, 'w', driver='GTiff',
                       width=xmax - xmin, height=ymax - ymin, count=src.count,
                       dtype=src.dtypes[0], crs=src.crs, transform=new_transform) as dst:
        dst.write(quarter_data)

# And convert to image
raster_data_set = gdal.Open(out_path)

raster_band_1 = raster_data_set.GetRasterBand(1) # red channel
raster_band_2 = raster_data_set.GetRasterBand(2) # green channel
raster_band_3 = raster_data_set.GetRasterBand(3) # blue channel

# Convert to array image
raster_b1 = raster_band_1.ReadAsArray()
raster_b2 = raster_band_2.ReadAsArray()
raster_b3 = raster_band_3.ReadAsArray()

# Stack to image
raster_img = np.dstack((raster_b1, raster_b2, raster_b3))

# Save image to file
plt.imsave(out_img, raster_img.astype(np.uint8), cmap='gray', format='png') # Convert to uint8

8785.2 10976.4 13177.800000000001 13720.5


In [ ]:
result = get_sliced_prediction(
    out_img, # path to subset image we created
    detection_model,
    slice_height = 300,
    slice_width = 300,
    overlap_height_ratio = 0.2,
    overlap_width_ratio = 0.2,
)

Performing prediction on 228 number of slices.


In [ ]:
result.export_visuals(export_dir="/content/drive/MyDrive/Master Geo Information Science/Internship/Data/Test rasters/", hide_labels=True)

Image(os.path.join("/content/drive/MyDrive/Master Geo Information Science/Internship/Data/Test rasters/", "prediction_visual.png"))

Output hidden; open in https://colab.research.google.com to view.

### Using 'result' object we can convert it to shapefiles like we did in the Detectron2 notebook